<a href="https://colab.research.google.com/github/SarahVSargent/stock_signals_regressions/blob/main/Sloan_Accurals_Analomaly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#author= SarahVSargent

In [ ]:
#### Import and Install Tools ###

!pip install pandasql

import os

from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split

from statsmodels.tsa.stattools import adfuller

from statsmodels.tsa.seasonal import seasonal_decompose

from statsmodels.tsa.arima_model import ARIMA

from sklearn.metrics import mean_squared_error, mean_absolute_error

import math

import pandas as pd

import statsmodels.api as sm

import statsmodels.formula.api as sm

import numpy as np

import matplotlib.pyplot as plt

from sqlite3 import connect

from scipy.stats.mstats import winsorize, ks_2samp

import seaborn as sns

import pandasql as ps

conn = connect(':memory:', timeout = 10) #connect to convert between pd dataframe and sql

#helpful to not have to copy paste everything eveyrme a new table is made
filePath = "/content/drive/MyDrive/Exeter google drive /internships jobs/FIMA '23/Sloane Accruals"

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26771 sha256=b7b284cc7a358b7fa35c371a2667f049a3bdeaf04fb098e6817c0655d9c6d7c3
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql


In [ ]:
#what is force remount? have to re mount drive each time its used
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)


Mounted at /content/gdrive


In [ ]:
file= "/content/gdrive/MyDrive/Exeter google drive /internships jobs/FIMA '23/Sloane Accruals/CRSPMonthly1990Through2022.csv"
crsp1990To2022= pd.read_csv(file)
print(crsp1990To2022.columns)
print(crsp1990To2022)

<ipython-input-4-c65205eb0b99>:2: DtypeWarning: Columns (5,9,18,19,49,58) have mixed types. Specify dtype option on import or set low_memory=False.
  crsp1990To2022= pd.read_csv(file)


Index(['PERMNO', 'date', 'NAMEENDT', 'SHRCD', 'EXCHCD', 'SICCD', 'NCUSIP',
       'TICKER', 'COMNAM', 'SHRCLS', 'TSYMBOL', 'NAICS', 'PRIMEXCH', 'TRDSTAT',
       'SECSTAT', 'PERMCO', 'ISSUNO', 'HEXCD', 'HSICCD', 'CUSIP', 'DCLRDT',
       'DLAMT', 'DLPDT', 'DLSTCD', 'NEXTDT', 'PAYDT', 'RCRDDT', 'SHRFLG',
       'HSICMG', 'HSICIG', 'DISTCD', 'DIVAMT', 'FACPR', 'FACSHR', 'ACPERM',
       'ACCOMP', 'SHRENDDT', 'NWPERM', 'DLRETX', 'DLPRC', 'DLRET', 'TRTSCD',
       'NMSIND', 'MMCNT', 'NSDINX', 'BIDLO', 'ASKHI', 'PRC', 'VOL', 'RET',
       'BID', 'ASK', 'SHROUT', 'CFACPR', 'CFACSHR', 'ALTPRC', 'SPREAD',
       'ALTPRCDT', 'RETX', 'vwretd', 'vwretx', 'ewretd', 'ewretx', 'sprtrn'],
      dtype='object')
         PERMNO      date  NAMEENDT  SHRCD  EXCHCD   SICCD    NCUSIP TICKER  \
0         10001  19900131       NaN   11.0     3.0    4920  39040610   GFGC   
1         10001  19900228       NaN   11.0     3.0    4920  39040610   GFGC   
2         10001  19900330       NaN   11.0     3.0    4920

In [ ]:
 ### Clean CRSP DataFrame ##

#Parse/select relevant variables
crsp1990To2022= crsp1990To2022.filter(['TICKER', 'date', 'SHRCD', 'EXCHCD', 'SICCD', 'SHROUT', 'ALTPRC', 'RET', 'DLRET', 'DLSTCD'])


print(crsp1990To2022)

        TICKER      date  SHRCD  EXCHCD   SICCD     SHROUT      ALTPRC  \
0         GFGC  19900131   11.0     3.0    4920     1022.0    -9.93750   
1         GFGC  19900228   11.0     3.0    4920     1022.0    -9.87500   
2         GFGC  19900330   11.0     3.0    4920     1027.0    -9.87500   
3         GFGC  19900430   11.0     3.0    4920     1027.0    -9.87500   
4         GFGC  19900531   11.0     3.0    4920     1027.0     9.75000   
...        ...       ...    ...     ...     ...        ...         ...   
2975236   TSLA  20211130   11.0     3.0  9999.0  1004265.0  1144.76001   
2975237   TSLA  20211231   11.0     3.0  9999.0  1033534.0  1056.78003   
2975238   TSLA  20220131   11.0     3.0  9999.0  1033534.0   936.71997   
2975239   TSLA  20220228   11.0     3.0  9999.0  1033534.0   870.42999   
2975240   TSLA  20220331   11.0     3.0  9999.0  1033534.0  1077.59998   

               RET DLRET  DLSTCD  
0        -0.018519   NaN     NaN  
1        -0.006289   NaN     NaN  
2     

In [ ]:

#Change column names
crsp1990To2022.columns= ['tic', 'date', 'shrcd', 'exchcd', 'siccd', 'shrout', 'altprc', 'ret', 'dlret', 'dlstcd']


print(crsp1990To2022)

          tic      date  shrcd  exchcd   siccd     shrout      altprc  \
0        GFGC  19900131   11.0     3.0    4920     1022.0    -9.93750   
1        GFGC  19900228   11.0     3.0    4920     1022.0    -9.87500   
2        GFGC  19900330   11.0     3.0    4920     1027.0    -9.87500   
3        GFGC  19900430   11.0     3.0    4920     1027.0    -9.87500   
4        GFGC  19900531   11.0     3.0    4920     1027.0     9.75000   
...       ...       ...    ...     ...     ...        ...         ...   
2975236  TSLA  20211130   11.0     3.0  9999.0  1004265.0  1144.76001   
2975237  TSLA  20211231   11.0     3.0  9999.0  1033534.0  1056.78003   
2975238  TSLA  20220131   11.0     3.0  9999.0  1033534.0   936.71997   
2975239  TSLA  20220228   11.0     3.0  9999.0  1033534.0   870.42999   
2975240  TSLA  20220331   11.0     3.0  9999.0  1033534.0  1077.59998   

               ret dlret  dlstcd  
0        -0.018519   NaN     NaN  
1        -0.006289   NaN     NaN  
2         0.012658

In [ ]:
#Convert variable types

#correction apply to astype
crsp1990To2022['tic']= crsp1990To2022['tic'].astype(str) #firm identifier


In [ ]:

crsp1990To2022['date'] = pd.to_datetime(crsp1990To2022['date'], format='%Y%m%d')


In [ ]:

crsp1990To2022['shrcd']= pd.to_numeric(crsp1990To2022['shrcd'], errors= 'coerce') #exchange code
crsp1990To2022['exchcd']= pd.to_numeric(crsp1990To2022['exchcd'], errors= 'coerce') #industry code
crsp1990To2022['siccd']= pd.to_numeric(crsp1990To2022['siccd'], errors= 'coerce') #share outstanding (thousands)
crsp1990To2022['shrout']= pd.to_numeric(crsp1990To2022['shrout'],errors= 'coerce') #last traded price in a month
crsp1990To2022['altprc'] = pd.to_numeric(crsp1990To2022['altprc'], errors= 'coerce') #return
crsp1990To2022['ret'] = pd.to_numeric(crsp1990To2022['ret'], errors= 'coerce') #delisting return
crsp1990To2022['dlret'] = pd.to_numeric(crsp1990To2022['dlret'], errors= 'coerce') #delisting code
crsp1990To2022['dlstcd'] = pd.to_numeric(crsp1990To2022['dlstcd'], errors= 'coerce')#delisting code

#Only keep dates after 2010

crsp2018To2022= crsp1990To2022[crsp1990To2022['date']>= "2018-01-01"]

#only keep US-based common stocks
crsp2018To2022= crsp2018To2022[(crsp2018To2022['shrcd'] == 10) |(crsp2018To2022['shrcd'] == 11)]

#Only keep NYSE, AMEX, and NASDAQ exchanges
crsp2018To2022= crsp2018To2022[(crsp2018To2022['exchcd'] == 1) |(crsp2018To2022['exchcd'] == 2) |(crsp2018To2022['exchcd'] == 3)| (crsp2018To2022['exchcd'] == 31)| (crsp2018To2022['exchcd'] == 32)| (crsp2018To2022['exchcd'] == 33)]

#Convert SICCD to 2-digit level
#floor truncates the product of decimal operation
crsp2018To2022['siccd'] = np.floor(crsp2018To2022['siccd'] / 100)

print(crsp2018To2022)

          tic       date  shrcd  exchcd  siccd     shrout      altprc  \
2363     JJSF 2018-01-31   11.0     3.0   20.0    18678.0   138.44000   
2364     JJSF 2018-02-28   11.0     3.0   20.0    18678.0   134.33000   
2365     JJSF 2018-03-29   11.0     3.0   20.0    18697.0   136.56000   
2366     JJSF 2018-04-30   11.0     3.0   20.0    18702.0   137.41000   
2367     JJSF 2018-05-31   11.0     3.0   20.0    18702.0   141.62000   
...       ...        ...    ...     ...    ...        ...         ...   
2975236  TSLA 2021-11-30   11.0     3.0   99.0  1004265.0  1144.76001   
2975237  TSLA 2021-12-31   11.0     3.0   99.0  1033534.0  1056.78003   
2975238  TSLA 2022-01-31   11.0     3.0   99.0  1033534.0   936.71997   
2975239  TSLA 2022-02-28   11.0     3.0   99.0  1033534.0   870.42999   
2975240  TSLA 2022-03-31   11.0     3.0   99.0  1033534.0  1077.59998   

              ret  dlret  dlstcd  
2363    -0.088191    NaN     NaN  
2364    -0.029688    NaN     NaN  
2365     0.019951 

In [ ]:

#Adjust delisting returns
crsp2018To2022['ret_adj']= ""

# for i in crsp2018To2022.index:

#   if (pd.isnull(crsp2018To2022['dlstcd'][i])) or (crsp2018To2022['dlstcd'][i] == 180):
#     crsp2018To2022['ret_adj'][i]=crsp2018To2022['ret'][i]

#   elif(pd.notnull(crsp2018To2022['dlstcd'][i])) and (pd.notnull(crsp2018To2022['dlret'][i])):
#     crsp2018To2022['ret_adj'][i]= crsp2018To2022['dlret'][i]

# #equal to any of the [] numbers
#   elif(551 <= crsp2018To2022['dlstcd'][1] <= 574) or (crsp2018To2022['dlstcd'][i] in [500, 520, 500, 584]):
#     crsp2018To2022['ret adj'][i]= -0.3

#   else:
#     crsp2018To2022['ret_adj'][i] = -1

#optimized and vectorized code for if, elif , else statement
#conditions [abc] choices [xyz]
#np.select(conditions, choices, default= else condition) is the key
conditions = [
    (crsp2018To2022['dlstcd'].isnull()) | (crsp2018To2022['dlstcd'] == 180),
    (crsp2018To2022['dlstcd'].notnull()) & (crsp2018To2022['dlret'].notnull()),
    (crsp2018To2022['dlstcd'].between(551, 574)) | (crsp2018To2022['dlstcd'].isin([500, 520, 584]))]

choices = [
    crsp2018To2022['ret'],
    crsp2018To2022['dlret'],
    -0.3]

crsp2018To2022['ret_adj'] = np.select(conditions, choices, default=-1)

print(crsp2018To2022)

          tic       date  shrcd  exchcd  siccd     shrout      altprc  \
2363     JJSF 2018-01-31   11.0     3.0   20.0    18678.0   138.44000   
2364     JJSF 2018-02-28   11.0     3.0   20.0    18678.0   134.33000   
2365     JJSF 2018-03-29   11.0     3.0   20.0    18697.0   136.56000   
2366     JJSF 2018-04-30   11.0     3.0   20.0    18702.0   137.41000   
2367     JJSF 2018-05-31   11.0     3.0   20.0    18702.0   141.62000   
...       ...        ...    ...     ...    ...        ...         ...   
2975236  TSLA 2021-11-30   11.0     3.0   99.0  1004265.0  1144.76001   
2975237  TSLA 2021-12-31   11.0     3.0   99.0  1033534.0  1056.78003   
2975238  TSLA 2022-01-31   11.0     3.0   99.0  1033534.0   936.71997   
2975239  TSLA 2022-02-28   11.0     3.0   99.0  1033534.0   870.42999   
2975240  TSLA 2022-03-31   11.0     3.0   99.0  1033534.0  1077.59998   

              ret  dlret  dlstcd   ret_adj  
2363    -0.088191    NaN     NaN -0.088191  
2364    -0.029688    NaN     NaN 

In [ ]:

crsp2018To2022.drop('dlret', inplace=True, axis= 1)
crsp2018To2022.drop('dlstcd', inplace=True, axis= 1)

print(crsp2018To2022)

          tic       date  shrcd  exchcd  siccd     shrout      altprc  \
2363     JJSF 2018-01-31   11.0     3.0   20.0    18678.0   138.44000   
2364     JJSF 2018-02-28   11.0     3.0   20.0    18678.0   134.33000   
2365     JJSF 2018-03-29   11.0     3.0   20.0    18697.0   136.56000   
2366     JJSF 2018-04-30   11.0     3.0   20.0    18702.0   137.41000   
2367     JJSF 2018-05-31   11.0     3.0   20.0    18702.0   141.62000   
...       ...        ...    ...     ...    ...        ...         ...   
2975236  TSLA 2021-11-30   11.0     3.0   99.0  1004265.0  1144.76001   
2975237  TSLA 2021-12-31   11.0     3.0   99.0  1033534.0  1056.78003   
2975238  TSLA 2022-01-31   11.0     3.0   99.0  1033534.0   936.71997   
2975239  TSLA 2022-02-28   11.0     3.0   99.0  1033534.0   870.42999   
2975240  TSLA 2022-03-31   11.0     3.0   99.0  1033534.0  1077.59998   

              ret   ret_adj  
2363    -0.088191 -0.088191  
2364    -0.029688 -0.029688  
2365     0.019951  0.019951  
236

clean compustat

In [ ]:
### Import Compustat CSV ###

file = "/content/gdrive/MyDrive/Exeter google drive /internships jobs/FIMA '23/Sloane Accruals/CompustatAnnual1950to2022.csv"

columnsToUse = ['tic', 'datadate', 'fyear', 'indfmt', 'datafmt', 'che', 'act', 'lct', 'at', 'dp', 'ib', 'dlc', 'txp', 'oiadp']
compustat1950To2022= pd.read_csv(file,infer_datetime_format=True,  usecols= columnsToUse)[columnsToUse]

print(compustat1950To2022)
print(compustat1950To2022.columns)

          tic  datadate   fyear indfmt datafmt       che        act  \
0        AE.2  19611231  1961.0   INDL     STD       NaN        NaN   
1        AE.2  19621231  1962.0   INDL     STD       NaN        NaN   
2        AE.2  19631231  1963.0   INDL     STD       NaN      0.408   
3        AE.2  19641231  1964.0   INDL     STD     0.269      0.718   
4        AE.2  19651231  1965.0   INDL     STD     0.031      0.725   
...       ...       ...     ...    ...     ...       ...        ...   
591305  IVCGF  20201231  2020.0   INDL     STD   558.000  12100.000   
591306  IVCGF  20211231  2021.0   INDL     STD  1020.181  12385.477   
591307  DTRUY  20191231  2019.0   INDL     STD  6532.226  31890.187   
591308  DTRUY  20201231  2020.0   INDL     STD  9012.671  30632.474   
591309  DTRUY  20211231  2021.0   INDL     STD  8358.207  31787.117   

              lct         at        dp        ib        dlc      txp     oiadp  
0             NaN        NaN       NaN     0.050        NaN      N

In [ ]:
#5

### Clean Compustat Dataframe ###


In [ ]:

#Change column names

compustat1950To2022.columns= ['tic', 'datadate', 'fyear', 'indfmt', 'datafmt', 'cash', 'ca', 'cl', 'ta', 'dep', 'ib', 'std', 'tp', 'oiadp']


In [ ]:
compustat1950To2022['tic']= compustat1950To2022['tic'].astype(str) #firm identifion

compustat1950To2022['datadate'] = pd.to_datetime(compustat1950To2022['datadate'], format= '%Y%m%d') #date of data

compustat1950To2022['fyear'] = pd.to_numeric(compustat1950To2022['fyear'], errors= 'coerce') #fiscal year of data

compustat1950To2022['indfmt']= compustat1950To2022['indfmt'].astype(str) #industry format

compustat1950To2022['datafmt']= compustat1950To2022['datafmt'].astype(str) #data fat

compustat1950To2022['cash']= pd.to_numeric(compustat1950To2022['cash'], errors= 'coerce') #cash and short-term investements

compustat1950To2022['ca'] = pd.to_numeric(compustat1950To2022['ca'], errors= 'coerce') #current assets

compustat1950To2022['cl'] = pd.to_numeric(compustat1950To2022['cl'], errors= 'coerce') #current liabilities

compustat1950To2022['ta']= pd.to_numeric(compustat1950To2022['ta'],errors= 'coerce') #total assets

compustat1950To2022['dep']= pd.to_numeric(compustat1950To2022['dep'], errors= 'coerce') #depreciation and amortization compustat1950To2022['ib] = pd.to numeric(compustat1950702822[1b1, errors coerce") #income before extraordinary items

compustat1950To2022['std']= pd.to_numeric(compustat1950To2022['std'], errors= 'coerce') #debt in current liabilities

compustat1950To2022['tp'] = pd.to_numeric(compustat1950To2022['tp'], errors= 'coerce') #income taxes payable

compustat1950To2022['oiadp'] = pd.to_numeric(compustat1950To2022['oiadp'], errors= 'coerce') #operating income after depreciation


In [ ]:
print(compustat1950To2022)
print(compustat1950To2022.columns)

          tic   datadate   fyear indfmt datafmt      cash         ca  \
0        AE.2 1961-12-31  1961.0   INDL     STD       NaN        NaN   
1        AE.2 1962-12-31  1962.0   INDL     STD       NaN        NaN   
2        AE.2 1963-12-31  1963.0   INDL     STD       NaN      0.408   
3        AE.2 1964-12-31  1964.0   INDL     STD     0.269      0.718   
4        AE.2 1965-12-31  1965.0   INDL     STD     0.031      0.725   
...       ...        ...     ...    ...     ...       ...        ...   
591305  IVCGF 2020-12-31  2020.0   INDL     STD   558.000  12100.000   
591306  IVCGF 2021-12-31  2021.0   INDL     STD  1020.181  12385.477   
591307  DTRUY 2019-12-31  2019.0   INDL     STD  6532.226  31890.187   
591308  DTRUY 2020-12-31  2020.0   INDL     STD  9012.671  30632.474   
591309  DTRUY 2021-12-31  2021.0   INDL     STD  8358.207  31787.117   

               cl         ta       dep        ib        std       tp     oiadp  
0             NaN        NaN       NaN     0.050      

In [ ]:

#Only keep data after 2017


#crsp2018To2022= crsp1990To2022[crsp1990To2022['date']>= "2018-01-01"]
compustat2018To2022= compustat1950To2022[compustat1950To2022['datadate']>= "2017-01-01"]

#Only keep correct formats

compustat2018To2022= compustat2018To2022[compustat2018To2022['indfmt']== "INDL"]
compustat2018To2022= compustat2018To2022[compustat2018To2022['datafmt']== "STD"]

#Drop Infinite/null values

compustat2018To2022.replace([np.inf, np.inf], np.nan, inplace=True)

compustat2018To2022= compustat2018To2022.dropna()

#reindex

compustat2018To2022= compustat2018To2022.reset_index()

compustat2018To2022.drop('index', inplace=True, axis = 1)

print(compustat2018To2022)
print(compustat2018To2022.columns)

         tic   datadate   fyear indfmt datafmt      cash         ca  \
0        AIR 2017-05-31  2016.0   INDL     STD    10.300    888.500   
1        AIR 2018-05-31  2017.0   INDL     STD    41.600    942.700   
2        AIR 2019-05-31  2018.0   INDL     STD    41.100    952.500   
3        AIR 2020-05-31  2019.0   INDL     STD   424.700   1438.700   
4        AIR 2021-05-31  2020.0   INDL     STD    60.200    937.000   
...      ...        ...     ...    ...     ...       ...        ...   
32179  IVCGF 2019-12-31  2019.0   INDL     STD   468.000  11570.000   
32180  IVCGF 2020-12-31  2020.0   INDL     STD   558.000  12100.000   
32181  DTRUY 2019-12-31  2019.0   INDL     STD  6532.226  31890.187   
32182  DTRUY 2020-12-31  2020.0   INDL     STD  9012.671  30632.474   
32183  DTRUY 2021-12-31  2021.0   INDL     STD  8358.207  31787.117   

              cl         ta       dep        ib        std       tp     oiadp  
0        335.100   1504.100    71.000    50.200      2.000   12.300

In [ ]:

#Calculate change delta) in variables

compustat2018To2022['delta_cash']= compustat2018To2022.groupby(['tic'])['cash'].diff()

compustat2018To2022['delta_ca']= compustat2018To2022.groupby(['tic'])['ca'].diff()

compustat2018To2022['delta_cl']= compustat2018To2022.groupby(['tic'])['cl'].diff()

compustat2018To2022['delta_std']= compustat2018To2022.groupby(['tic'])['std'].diff()

compustat2018To2022['delta_tp']= compustat2018To2022.groupby(['tic'])['tp'].diff()

#Drup null values (first year no difference computed)

compustat2018To2022= compustat2018To2022.dropna()


In [ ]:

print(compustat2018To2022)
print(compustat2018To2022.columns)

         tic   datadate   fyear indfmt datafmt      cash         ca  \
1        AIR 2018-05-31  2017.0   INDL     STD    41.600    942.700   
2        AIR 2019-05-31  2018.0   INDL     STD    41.100    952.500   
3        AIR 2020-05-31  2019.0   INDL     STD   424.700   1438.700   
4        AIR 2021-05-31  2020.0   INDL     STD    60.200    937.000   
5        AIR 2022-05-31  2021.0   INDL     STD    58.900   1007.200   
...      ...        ...     ...    ...     ...       ...        ...   
32176   KARO 2021-02-28  2020.0   INDL     STD    65.057     87.432   
32177   KARO 2022-02-28  2021.0   INDL     STD    47.427     72.275   
32180  IVCGF 2020-12-31  2020.0   INDL     STD   558.000  12100.000   
32182  DTRUY 2020-12-31  2020.0   INDL     STD  9012.671  30632.474   
32183  DTRUY 2021-12-31  2021.0   INDL     STD  8358.207  31787.117   

              cl         ta       dep        ib        std       tp     oiadp  \
1        333.300   1524.700    40.500    73.700      0.000    0.00

In [ ]:



#Calculate accruals

compustat2018To2022['accruals']= (compustat2018To2022['delta_ca']- compustat2018To2022['delta_cash'] - compustat2018To2022['delta_cl'] - compustat2018To2022['delta_std'] - compustat2018To2022['delta_tp'])- compustat2018To2022['dep']

compustat2018To2022['accruals_subcomp_1']= compustat2018To2022['delta_ca']- compustat2018To2022 ['delta_cash']

compustat2018To2022['accruals_subcomp_2'] = -(compustat2018To2022['delta_cl']- compustat2018To2022['delta_std']- compustat2018To2022['delta_tp'])

compustat2018To2022['accruals_subcomp_3']= -(compustat2018To2022['dep'])


print(compustat2018To2022)
print(compustat2018To2022.columns)

         tic   datadate   fyear indfmt datafmt      cash         ca  \
1        AIR 2018-05-31  2017.0   INDL     STD    41.600    942.700   
2        AIR 2019-05-31  2018.0   INDL     STD    41.100    952.500   
3        AIR 2020-05-31  2019.0   INDL     STD   424.700   1438.700   
4        AIR 2021-05-31  2020.0   INDL     STD    60.200    937.000   
5        AIR 2022-05-31  2021.0   INDL     STD    58.900   1007.200   
...      ...        ...     ...    ...     ...       ...        ...   
32176   KARO 2021-02-28  2020.0   INDL     STD    65.057     87.432   
32177   KARO 2022-02-28  2021.0   INDL     STD    47.427     72.275   
32180  IVCGF 2020-12-31  2020.0   INDL     STD   558.000  12100.000   
32182  DTRUY 2020-12-31  2020.0   INDL     STD  9012.671  30632.474   
32183  DTRUY 2021-12-31  2021.0   INDL     STD  8358.207  31787.117   

              cl         ta       dep  ...     oiadp  delta_cash  delta_ca  \
1        333.300   1524.700    40.500  ...    86.000      31.300    5

In [ ]:
#5
#Calculate average total assets

print(compustat2018To2022)

         tic   datadate   fyear indfmt datafmt      cash         ca  \
1        AIR 2018-05-31  2017.0   INDL     STD    41.600    942.700   
2        AIR 2019-05-31  2018.0   INDL     STD    41.100    952.500   
3        AIR 2020-05-31  2019.0   INDL     STD   424.700   1438.700   
4        AIR 2021-05-31  2020.0   INDL     STD    60.200    937.000   
5        AIR 2022-05-31  2021.0   INDL     STD    58.900   1007.200   
...      ...        ...     ...    ...     ...       ...        ...   
32176   KARO 2021-02-28  2020.0   INDL     STD    65.057     87.432   
32177   KARO 2022-02-28  2021.0   INDL     STD    47.427     72.275   
32180  IVCGF 2020-12-31  2020.0   INDL     STD   558.000  12100.000   
32182  DTRUY 2020-12-31  2020.0   INDL     STD  9012.671  30632.474   
32183  DTRUY 2021-12-31  2021.0   INDL     STD  8358.207  31787.117   

              cl         ta       dep  ...     oiadp  delta_cash  delta_ca  \
1        333.300   1524.700    40.500  ...    86.000      31.300    5

In [ ]:
avgATDF = compustat2018To2022.groupby(['tic'], as_index = False)['ta'].mean()

avgATDF.columns = ['tic', 'avg_at']


In [ ]:

compustat2018To2022= compustat2018To2022.merge(avgATDF, on= 'tic', how= 'left')

print(compustat2018To2022)

         tic   datadate   fyear indfmt datafmt      cash         ca  \
0        AIR 2018-05-31  2017.0   INDL     STD    41.600    942.700   
1        AIR 2019-05-31  2018.0   INDL     STD    41.100    952.500   
2        AIR 2020-05-31  2019.0   INDL     STD   424.700   1438.700   
3        AIR 2021-05-31  2020.0   INDL     STD    60.200    937.000   
4        AIR 2022-05-31  2021.0   INDL     STD    58.900   1007.200   
...      ...        ...     ...    ...     ...       ...        ...   
24345   KARO 2021-02-28  2020.0   INDL     STD    65.057     87.432   
24346   KARO 2022-02-28  2021.0   INDL     STD    47.427     72.275   
24347  IVCGF 2020-12-31  2020.0   INDL     STD   558.000  12100.000   
24348  DTRUY 2020-12-31  2020.0   INDL     STD  9012.671  30632.474   
24349  DTRUY 2021-12-31  2021.0   INDL     STD  8358.207  31787.117   

              cl         ta       dep  ...  delta_cash  delta_ca  delta_cl  \
0        333.300   1524.700    40.500  ...      31.300    54.200    -

In [ ]:

#Calculate earnings component

compustat2018To2022['earnings_comp']= compustat2018To2022['oiadp'] / compustat2018To2022['avg_at']

#Calculate accruals component

compustat2018To2022['accruals_comp'] = compustat2018To2022['accruals']/ compustat2018To2022['avg_at']

#calculate cash flow component

compustat2018To2022['cash_flows_comp'] = compustat2018To2022['earnings_comp']- compustat2018To2022['accruals_comp']

#Standradize sub-components
#shorthand for divide left by right and replace left value
compustat2018To2022['accruals_subcomp_1'] /= compustat2018To2022['avg_at']

compustat2018To2022['accruals_subcomp_2'] /= compustat2018To2022['avg_at']

compustat2018To2022['accruals_subcomp_3'] /= compustat2018To2022['avg_at']

#Drop infinite/null values

compustat2018To2022.replace([np.inf, np.inf], np.nan, inplace=True)
compustat2018To2022= compustat2018To2022.dropna()

#Reindex

compustat2018To2022= compustat2018To2022.reset_index()

compustat2018To2022.drop(['index'], inplace= True, axis= 1)

print(compustat2018To2022)

         tic   datadate   fyear indfmt datafmt      cash         ca  \
0        AIR 2018-05-31  2017.0   INDL     STD    41.600    942.700   
1        AIR 2019-05-31  2018.0   INDL     STD    41.100    952.500   
2        AIR 2020-05-31  2019.0   INDL     STD   424.700   1438.700   
3        AIR 2021-05-31  2020.0   INDL     STD    60.200    937.000   
4        AIR 2022-05-31  2021.0   INDL     STD    58.900   1007.200   
...      ...        ...     ...    ...     ...       ...        ...   
24318   KARO 2021-02-28  2020.0   INDL     STD    65.057     87.432   
24319   KARO 2022-02-28  2021.0   INDL     STD    47.427     72.275   
24320  IVCGF 2020-12-31  2020.0   INDL     STD   558.000  12100.000   
24321  DTRUY 2020-12-31  2020.0   INDL     STD  9012.671  30632.474   
24322  DTRUY 2021-12-31  2021.0   INDL     STD  8358.207  31787.117   

              cl         ta       dep  ...  delta_std  delta_tp   accruals  \
0        333.300   1524.700    40.500  ...     -2.000   -12.300     -

In [ ]:
compustat2018To2022['date']= compustat2018To2022['datadate']
print(compustat2018To2022)

         tic   datadate   fyear indfmt datafmt      cash         ca  \
0        AIR 2018-05-31  2017.0   INDL     STD    41.600    942.700   
1        AIR 2019-05-31  2018.0   INDL     STD    41.100    952.500   
2        AIR 2020-05-31  2019.0   INDL     STD   424.700   1438.700   
3        AIR 2021-05-31  2020.0   INDL     STD    60.200    937.000   
4        AIR 2022-05-31  2021.0   INDL     STD    58.900   1007.200   
...      ...        ...     ...    ...     ...       ...        ...   
24318   KARO 2021-02-28  2020.0   INDL     STD    65.057     87.432   
24319   KARO 2022-02-28  2021.0   INDL     STD    47.427     72.275   
24320  IVCGF 2020-12-31  2020.0   INDL     STD   558.000  12100.000   
24321  DTRUY 2020-12-31  2020.0   INDL     STD  9012.671  30632.474   
24322  DTRUY 2021-12-31  2021.0   INDL     STD  8358.207  31787.117   

              cl         ta       dep  ...  delta_tp   accruals  \
0        333.300   1524.700    40.500  ...   -12.300     -1.500   
1        357.

In [ ]:
#6
#left join
mainDF= crsp2018To2022.merge(compustat2018To2022, how='left', on= ['tic', 'date'])
print(mainDF)

         tic       date  shrcd  exchcd  siccd     shrout      altprc  \
0       JJSF 2018-01-31   11.0     3.0   20.0    18678.0   138.44000   
1       JJSF 2018-02-28   11.0     3.0   20.0    18678.0   134.33000   
2       JJSF 2018-03-29   11.0     3.0   20.0    18697.0   136.56000   
3       JJSF 2018-04-30   11.0     3.0   20.0    18702.0   137.41000   
4       JJSF 2018-05-31   11.0     3.0   20.0    18702.0   141.62000   
...      ...        ...    ...     ...    ...        ...         ...   
193909  TSLA 2021-11-30   11.0     3.0   99.0  1004265.0  1144.76001   
193910  TSLA 2021-12-31   11.0     3.0   99.0  1033534.0  1056.78003   
193911  TSLA 2022-01-31   11.0     3.0   99.0  1033534.0   936.71997   
193912  TSLA 2022-02-28   11.0     3.0   99.0  1033534.0   870.42999   
193913  TSLA 2022-03-31   11.0     3.0   99.0  1033534.0  1077.59998   

             ret   ret_adj   datadate  ...  delta_std delta_tp accruals  \
0      -0.088191 -0.088191        NaT  ...        NaN      N

FFDF cleaning

In [ ]:

file = "/content/gdrive/MyDrive/Exeter google drive /internships jobs/FIMA '23/Sloane Accruals/F-F_Research_Data_5_Factors.csv"
ffDF= pd.read_csv(file)

print(ffDF)
print(ffDF.columns)

       Date  Mkt_RF   SMB    HML   RMW   CMA    RF
0    196307   -0.39 -0.44  -0.89  0.68 -1.23  0.27
1    196308    5.07 -0.75   1.68  0.36 -0.34  0.25
2    196309   -1.57 -0.55   0.08 -0.71  0.29  0.27
3    196310    2.53 -1.37  -0.14  2.80 -2.02  0.29
4    196311   -0.85 -0.89   1.81 -0.51  2.31  0.27
..      ...     ...   ...    ...   ...   ...   ...
702  202201   -6.25 -3.95  12.74  0.73  7.73  0.00
703  202202   -2.29  2.90   3.09 -2.12  2.99  0.00
704  202203    3.06 -2.14  -1.82 -1.32  3.24  0.00
705  202204   -9.45 -0.38   6.16  3.51  5.87  0.00
706  202205   -0.34  0.02   8.38  1.61  3.82  0.03

[707 rows x 7 columns]
Index(['Date', 'Mkt_RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF'], dtype='object')


In [ ]:

ffDF = ffDF[ffDF['Date'] >= 201801]
print(ffDF)

       Date  Mkt_RF   SMB    HML   RMW   CMA    RF
654  201801    5.58 -3.25  -1.36 -0.72 -1.05  0.11
655  201802   -3.65  0.33  -1.03  0.56 -2.40  0.11
656  201803   -2.35  3.59  -0.23 -0.49  0.02  0.12
657  201804    0.29  0.90   0.48 -2.41  1.28  0.14
658  201805    2.65  4.78  -3.13 -2.04 -1.43  0.14
659  201806    0.48  0.80  -2.33  0.82  0.23  0.14
660  201807    3.19 -1.94   0.45  1.52  0.37  0.16
661  201808    3.44  0.63  -3.98 -0.30 -2.66  0.16
662  201809    0.06 -2.49  -1.70  0.67  1.30  0.15
663  201810   -7.68 -4.46   3.43  0.98  3.60  0.19
664  201811    1.69 -0.77   0.26 -0.65  0.36  0.18
665  201812   -9.55 -2.90  -1.90 -0.04  0.22  0.19
666  201901    8.41  3.01  -0.44 -0.77 -1.51  0.21
667  201902    3.40  1.72  -2.68  0.21 -1.62  0.18
668  201903    1.10 -3.47  -4.05  0.81 -0.97  0.19
669  201904    3.97 -1.14   2.17  1.64 -2.20  0.21
670  201905   -6.94 -1.58  -2.37 -0.51  1.75  0.21
671  201906    6.93  0.36  -0.71  0.86 -0.43  0.18
672  201907    1.19 -1.87   0.5

In [ ]:

#Change column names

ffDF.columns = ['date', 'rm_minus_rf', 'smb', 'hml', 'rmw', 'cma', 'rf']
print(ffDF)

       date  rm_minus_rf   smb    hml   rmw   cma    rf
654  201801         5.58 -3.25  -1.36 -0.72 -1.05  0.11
655  201802        -3.65  0.33  -1.03  0.56 -2.40  0.11
656  201803        -2.35  3.59  -0.23 -0.49  0.02  0.12
657  201804         0.29  0.90   0.48 -2.41  1.28  0.14
658  201805         2.65  4.78  -3.13 -2.04 -1.43  0.14
659  201806         0.48  0.80  -2.33  0.82  0.23  0.14
660  201807         3.19 -1.94   0.45  1.52  0.37  0.16
661  201808         3.44  0.63  -3.98 -0.30 -2.66  0.16
662  201809         0.06 -2.49  -1.70  0.67  1.30  0.15
663  201810        -7.68 -4.46   3.43  0.98  3.60  0.19
664  201811         1.69 -0.77   0.26 -0.65  0.36  0.18
665  201812        -9.55 -2.90  -1.90 -0.04  0.22  0.19
666  201901         8.41  3.01  -0.44 -0.77 -1.51  0.21
667  201902         3.40  1.72  -2.68  0.21 -1.62  0.18
668  201903         1.10 -3.47  -4.05  0.81 -0.97  0.19
669  201904         3.97 -1.14   2.17  1.64 -2.20  0.21
670  201905        -6.94 -1.58  -2.37 -0.51  1.7

In [ ]:
### Clean FF DataFrame ###

#Optimize in chat gpt
#YASSSS this works too!!


ffDF['year'] = ""
ffDF['month'] = ""
ffDF['year']= ffDF['date'].apply(lambda x: str(x)[0:4])
ffDF['month']= ffDF['date'].apply(lambda x: str(x)[-2]+ str(x)[-1])

print(ffDF)


       date  rm_minus_rf   smb    hml   rmw   cma    rf  year month
654  201801         5.58 -3.25  -1.36 -0.72 -1.05  0.11  2018    01
655  201802        -3.65  0.33  -1.03  0.56 -2.40  0.11  2018    02
656  201803        -2.35  3.59  -0.23 -0.49  0.02  0.12  2018    03
657  201804         0.29  0.90   0.48 -2.41  1.28  0.14  2018    04
658  201805         2.65  4.78  -3.13 -2.04 -1.43  0.14  2018    05
659  201806         0.48  0.80  -2.33  0.82  0.23  0.14  2018    06
660  201807         3.19 -1.94   0.45  1.52  0.37  0.16  2018    07
661  201808         3.44  0.63  -3.98 -0.30 -2.66  0.16  2018    08
662  201809         0.06 -2.49  -1.70  0.67  1.30  0.15  2018    09
663  201810        -7.68 -4.46   3.43  0.98  3.60  0.19  2018    10
664  201811         1.69 -0.77   0.26 -0.65  0.36  0.18  2018    11
665  201812        -9.55 -2.90  -1.90 -0.04  0.22  0.19  2018    12
666  201901         8.41  3.01  -0.44 -0.77 -1.51  0.21  2019    01
667  201902         3.40  1.72  -2.68  0.21 -1.6

In [ ]:

#hange values to decimal

ffDF['rm_minus_rf'] /= 100

ffDF['smb']/= 100

ffDF['hml']/= 100

ffDF['rmw'] /= 100

ffDF['cma'] /= 100

ffDF['rf'] /= 100

print(ffDF)

       date  rm_minus_rf     smb     hml     rmw     cma      rf  year month
654  201801       0.0558 -0.0325 -0.0136 -0.0072 -0.0105  0.0011  2018    01
655  201802      -0.0365  0.0033 -0.0103  0.0056 -0.0240  0.0011  2018    02
656  201803      -0.0235  0.0359 -0.0023 -0.0049  0.0002  0.0012  2018    03
657  201804       0.0029  0.0090  0.0048 -0.0241  0.0128  0.0014  2018    04
658  201805       0.0265  0.0478 -0.0313 -0.0204 -0.0143  0.0014  2018    05
659  201806       0.0048  0.0080 -0.0233  0.0082  0.0023  0.0014  2018    06
660  201807       0.0319 -0.0194  0.0045  0.0152  0.0037  0.0016  2018    07
661  201808       0.0344  0.0063 -0.0398 -0.0030 -0.0266  0.0016  2018    08
662  201809       0.0006 -0.0249 -0.0170  0.0067  0.0130  0.0015  2018    09
663  201810      -0.0768 -0.0446  0.0343  0.0098  0.0360  0.0019  2018    10
664  201811       0.0169 -0.0077  0.0026 -0.0065  0.0036  0.0018  2018    11
665  201812      -0.0955 -0.0290 -0.0190 -0.0004  0.0022  0.0019  2018    12

In [ ]:
ffDFcheckpoint= ffDF

In [ ]:
ffDF= ffDFcheckpoint

In [ ]:

ffDF['date'] = pd.to_datetime(ffDF['date'], format= '%Y%m')
print(ffDF)

          date  rm_minus_rf     smb     hml     rmw     cma      rf  year  \
654 2018-01-01       0.0558 -0.0325 -0.0136 -0.0072 -0.0105  0.0011  2018   
655 2018-02-01      -0.0365  0.0033 -0.0103  0.0056 -0.0240  0.0011  2018   
656 2018-03-01      -0.0235  0.0359 -0.0023 -0.0049  0.0002  0.0012  2018   
657 2018-04-01       0.0029  0.0090  0.0048 -0.0241  0.0128  0.0014  2018   
658 2018-05-01       0.0265  0.0478 -0.0313 -0.0204 -0.0143  0.0014  2018   
659 2018-06-01       0.0048  0.0080 -0.0233  0.0082  0.0023  0.0014  2018   
660 2018-07-01       0.0319 -0.0194  0.0045  0.0152  0.0037  0.0016  2018   
661 2018-08-01       0.0344  0.0063 -0.0398 -0.0030 -0.0266  0.0016  2018   
662 2018-09-01       0.0006 -0.0249 -0.0170  0.0067  0.0130  0.0015  2018   
663 2018-10-01      -0.0768 -0.0446  0.0343  0.0098  0.0360  0.0019  2018   
664 2018-11-01       0.0169 -0.0077  0.0026 -0.0065  0.0036  0.0018  2018   
665 2018-12-01      -0.0955 -0.0290 -0.0190 -0.0004  0.0022  0.0019  2018   

In [ ]:

#Only keep dates after 2018
# ffDF['date'] = ffDF['date'].astype('datetime64[D]')
#one from slide
#ffDF = ffDF[ffDF['date'] >= 201801]

# #chatgpt
# ffDF['date'] = ffDF['date'].astype('datetime64[D]')
# ffDF = ffDF[ffDF['year'] >= pd.to_datetime('2018-01-01')]

# # ffDF['year']= pd.to_ffDF['year']
# ffDF['year'] = ffDF['year'].astype(int)
# ffDF = ffDF[ffDF['year'] >= 2018]


# print(ffDF)

In [ ]:

# ffDF['year'] = ""
# ffDF['month'] = ""

# for i in ffDF.index:
#   ffDF['year'][i] = int(str(ffDF['date'][i])[0:4]) #take first 4 digits of date (year)
#   ffDF['month'][1] = int(str(ffDF['date'][i])[-2] + str(ffDF['date'][i])[-1]) #take last 2 digits of date (month)



# # Convert the 'date' column to datetime format
# ffDF['date'] = pd.to_datetime(ffDF['date'])

# # Extract the 'year' and 'month' columns using vectorized operations
# ffDF['year'] = ffDF['date'].dt.year
# ffDF['month'] = ffDF['date'].dt.month

# #diving by 1000 for only the first digit
# # Assuming you want to keep the first 4 digits of the year column (as per the original code)
# ffDF['year'] = ffDF['year'] // 1000 * 1000

# # Assuming you want to keep the last 2 digits of the month column (as per the original code)
# ffDF['month'] = ffDF['month'] % 100

# # If you want to keep both year and month as strings
# ffDF['year'] = ffDF['year'].astype(str)
# ffDF['month'] = ffDF['month'].astype(str)




# ffDF['year'] = ffDF['date'].dt.year
# ffDF['month'] = ffDF['date'].dt.month


#Reindex
ffDF= ffDF.reset_index()
ffDF.drop('index', inplace= True, axis = 1)

print(ffDF)

         date  rm_minus_rf     smb     hml     rmw     cma      rf  year month
0  2018-01-01       0.0558 -0.0325 -0.0136 -0.0072 -0.0105  0.0011  2018    01
1  2018-02-01      -0.0365  0.0033 -0.0103  0.0056 -0.0240  0.0011  2018    02
2  2018-03-01      -0.0235  0.0359 -0.0023 -0.0049  0.0002  0.0012  2018    03
3  2018-04-01       0.0029  0.0090  0.0048 -0.0241  0.0128  0.0014  2018    04
4  2018-05-01       0.0265  0.0478 -0.0313 -0.0204 -0.0143  0.0014  2018    05
5  2018-06-01       0.0048  0.0080 -0.0233  0.0082  0.0023  0.0014  2018    06
6  2018-07-01       0.0319 -0.0194  0.0045  0.0152  0.0037  0.0016  2018    07
7  2018-08-01       0.0344  0.0063 -0.0398 -0.0030 -0.0266  0.0016  2018    08
8  2018-09-01       0.0006 -0.0249 -0.0170  0.0067  0.0130  0.0015  2018    09
9  2018-10-01      -0.0768 -0.0446  0.0343  0.0098  0.0360  0.0019  2018    10
10 2018-11-01       0.0169 -0.0077  0.0026 -0.0065  0.0036  0.0018  2018    11
11 2018-12-01      -0.0955 -0.0290 -0.0190 -0.0004  

In [ ]:
#Calculate annual market return for risk free rate
annualRmMinusRf = pd.DataFrame(index = [2018, 2019, 2020, 2021], columns = ['year', 'annual_rm_minus_rf'])
annualRmMinusRf['year'] = annualRmMinusRf.index

tempDF = ffDF.loc[0:11]
#basic rate calculation
tempDF['rm_minus_rf_compounded'] = (1 + tempDF['rm_minus_rf']).cumprod() - 1
annualRmMinusRf['annual_rm_minus_rf'][2018] = tempDF['rm_minus_rf_compounded'][11]

tempDF = ffDF.loc[12:23]
#basic rate calculation
tempDF['rm_minus_rf_compounded'] = (1 + tempDF['rm_minus_rf']).cumprod() - 1
annualRmMinusRf['annual_rm_minus_rf'][2019] = tempDF['rm_minus_rf_compounded'][23]


tempDF = ffDF.loc[24:35]
#basic rate calculation
tempDF['rm_minus_rf_compounded'] = (1 + tempDF['rm_minus_rf']).cumprod() - 1
annualRmMinusRf['annual_rm_minus_rf'][2020] = tempDF['rm_minus_rf_compounded'][35]


tempDF = ffDF.loc[36:47]
#basic rate calculation
tempDF['rm_minus_rf_compounded'] = (1 + tempDF['rm_minus_rf']).cumprod() - 1
annualRmMinusRf['annual_rm_minus_rf'][2021] = tempDF['rm_minus_rf_compounded'][47]

print(annualRmMinusRf)

      year annual_rm_minus_rf
2018  2018          -0.068096
2019  2019           0.277375
2020  2020           0.235311
2021  2021           0.235548


<ipython-input-37-61d270443e48>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tempDF['rm_minus_rf_compounded'] = (1 + tempDF['rm_minus_rf']).cumprod() - 1
<ipython-input-37-61d270443e48>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annualRmMinusRf['annual_rm_minus_rf'][2018] = tempDF['rm_minus_rf_compounded'][11]
<ipython-input-37-61d270443e48>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

Work with mainDF

In [ ]:
print(mainDF)

         tic       date  shrcd  exchcd  siccd     shrout      altprc  \
0       JJSF 2018-01-31   11.0     3.0   20.0    18678.0   138.44000   
1       JJSF 2018-02-28   11.0     3.0   20.0    18678.0   134.33000   
2       JJSF 2018-03-29   11.0     3.0   20.0    18697.0   136.56000   
3       JJSF 2018-04-30   11.0     3.0   20.0    18702.0   137.41000   
4       JJSF 2018-05-31   11.0     3.0   20.0    18702.0   141.62000   
...      ...        ...    ...     ...    ...        ...         ...   
193909  TSLA 2021-11-30   11.0     3.0   99.0  1004265.0  1144.76001   
193910  TSLA 2021-12-31   11.0     3.0   99.0  1033534.0  1056.78003   
193911  TSLA 2022-01-31   11.0     3.0   99.0  1033534.0   936.71997   
193912  TSLA 2022-02-28   11.0     3.0   99.0  1033534.0   870.42999   
193913  TSLA 2022-03-31   11.0     3.0   99.0  1033534.0  1077.59998   

             ret   ret_adj   datadate  ...  delta_std delta_tp accruals  \
0      -0.088191 -0.088191        NaT  ...        NaN      N

In [ ]:
print(mainDF.columns)

Index(['tic', 'date', 'shrcd', 'exchcd', 'siccd', 'shrout', 'altprc', 'ret',
       'ret_adj', 'datadate', 'fyear', 'indfmt', 'datafmt', 'cash', 'ca', 'cl',
       'ta', 'dep', 'ib', 'std', 'tp', 'oiadp', 'delta_cash', 'delta_ca',
       'delta_cl', 'delta_std', 'delta_tp', 'accruals', 'accruals_subcomp_1',
       'accruals_subcomp_2', 'accruals_subcomp_3', 'avg_at', 'earnings_comp',
       'accruals_comp', 'cash_flows_comp', 'rm_minus_rf', 'smb', 'hml', 'rmw',
       'cma', 'rf', 'year', 'month'],
      dtype='object')


In [ ]:
#Left join relevant FF (risk-free rate) data into main DataFrame
mainDF= mainDF.merge(ffDF, how= 'left', on= 'date')

print(mainDF)

         tic       date  shrcd  exchcd  siccd     shrout      altprc  \
0       JJSF 2018-01-31   11.0     3.0   20.0    18678.0   138.44000   
1       JJSF 2018-02-28   11.0     3.0   20.0    18678.0   134.33000   
2       JJSF 2018-03-29   11.0     3.0   20.0    18697.0   136.56000   
3       JJSF 2018-04-30   11.0     3.0   20.0    18702.0   137.41000   
4       JJSF 2018-05-31   11.0     3.0   20.0    18702.0   141.62000   
...      ...        ...    ...     ...    ...        ...         ...   
193909  TSLA 2021-11-30   11.0     3.0   99.0  1004265.0  1144.76001   
193910  TSLA 2021-12-31   11.0     3.0   99.0  1033534.0  1056.78003   
193911  TSLA 2022-01-31   11.0     3.0   99.0  1033534.0   936.71997   
193912  TSLA 2022-02-28   11.0     3.0   99.0  1033534.0   870.42999   
193913  TSLA 2022-03-31   11.0     3.0   99.0  1033534.0  1077.59998   

             ret   ret_adj   datadate  ...  accruals_comp cash_flows_comp  \
0      -0.088191 -0.088191        NaT  ...            NaN 

Make size deciles

In [ ]:
mainDFcheckpoint=mainDF

In [ ]:
mainDF= mainDFcheckpoint

In [ ]:
file = "/content/drive/MyDrive/Exeter google drive /internships jobs/FIMA '23/Ertimur replication/Compustat2018Through2022.csv"
columnsToUse= ['cshoq', 'prccq', 'tic', 'rdq']
marketvaluedf = pd.read_csv(file, usecols= columnsToUse)[columnsToUse]
print(marketvaluedf.columns)
print(marketvaluedf)

OSError: ignored

In [ ]:
marketvaluedf['tic']= marketvaluedf['tic'].astype(str)
marketvaluedf['rdq']= pd.to_numeric(marketvaluedf['rdq'],errors= 'coerce')
marketvaluedf['cshoq']= pd.to_numeric(marketvaluedf['cshoq'], errors= 'coerce')
marketvaluedf['prccq']= pd.to_numeric(marketvaluedf['prccq'],errors= 'coerce')
print(marketvaluedf)

In [ ]:
marketvaluedf2019= marketvaluedf[marketvaluedf['datadate'] >= "2019-01-01"]
print(marketvaluedf2019)

In [ ]:
mainDF= mainDF.merge(marketvaluedf2019, how= 'left', on= 'rdq')
print(mainDF)



In [ ]:
mainDF['market_value'] = mainDF['cshoq'] * mainDF['prccq']
print(mainDF)


In [ ]:
onerdq= mainDF

print(onerdq)

In [ ]:
print(onerdq.columns)

In [ ]:

#new df with only year and first market value
marketvaluestartyear= onerdq.groupby(['year', 'tic'], as_index= False)['market_value'].first()


print(marketvaluestartyear)
print(marketvaluestartyear.columns)

In [ ]:

#new column with size quantile for each firm, pct is percentile rank, using marketing value in a specific year
marketvaluestartyear['size_quantile']= marketvaluestartyear.groupby(['year'])['market_value'].rank(pct= True)


print(marketvaluestartyear)
print(marketvaluestartyear.columns)

In [ ]:

#quaantiles (percentiles) to deciles
marketvaluestartyear['size_decile']= np.ceil(marketvaluestartyear['size_quantile']/10)



print(marketvaluestartyear)
print(marketvaluestartyear.columns)


In [ ]:
#5
#left join merdge into maindf
mainDF= mainDF.merge(marketvaluestartyear, on= 'tic', 'year', how= 'left')

print(mainDF.columns)
print(mainDF)

In [ ]:

mainDF['date']= pd.to_datetime(mainDF['date'])
mainDF['ret']= pd.to_numeric(mainDF['ret'], errors = 'coerce')
mainDF['ret_adj']= pd.to_numeric(mainDF['ret_adj'], errors = 'coerce')
mainDF['size_decile']= pd.to_numeric(mainDF['size_decile'], errors = 'coerce')
mainDF['ret_minus_rf'] = mainDF['ret_adj'] - mainDF['rf']

#Add abnormal return data

print(mainDF)

In [ ]:
#reate an array for calulations, merge back

g= mainDF.groupby(['year', 'month', 'size decile'], as_index = False).apply(lambda x: np.average (x['ret_adj']))
g.columns = ['year', 'month', 'size_decile', 'avg_ret_on_size_decile']
print(g)



In [ ]:
#maybe switch order of on and how
mainDF= mainDF.merge(g, on= ['year', 'month', 'size_decile'], how = 'left')
mainDF['abn_ret'] = mainDF['ret_adj']-  mainDF['avg_ret_on_size_decile']
print(mainDF)

In [ ]:
mainDF.drop('index', inplace= True, axis= 1)
print(mainDF)

In [ ]:
#Store mainDF for future use
tempMain= mainDF

print(mainDF)

Table 1A

In [ ]:
### Replicate Table 1A ###
mainDF = tempMain

In [ ]:

#Generate Dataframe with non-null components (end of fiscal year data)
mainOnFYearEnd = mainDF.dropna()
mainOnFYearEnd['accruals_comp_quantile'] = mainOnFYearEnd ['accruals_comp'].rank(pct = True)
mainOnFYearEnd['accruals_comp_decile']= np.ceil(mainOnFYearEnd['accruals_comp_quantile'] * 10)

In [ ]:

#Generate data for firms with December fiscal-year ends (for CAPM regressions)
mainOnFYearEndDecOnly = mainOnFYearEnd[mainOnFYearEnd['month']==12]

In [ ]:
#Generate summary statistics (Table 1A)
table1A= pd.Dataframe(index = ['accruals_mean', 'accruals_median', 'cash_flows_mean', 'cash_flows_median', 'earnings_mean', 'earnings_median'], columns = range(1, 11))
for i in table1A.columns:
  tempDF = mainOnFYearEnd[mainOnFYearEnd['accruals_comp decile'] == i]
  table1A[i]['accruals_mean mean' ] = tempDF['accruals_comp'].mean ()
  table1A[i]['accruals_median'] = tempDF['accruals_comp'].median()
  table1A[i]['cash_flows_mean']= tempDF ['cash_flows_comp'].mean()
  table1A[i]['cash_flows_median'] = tempDF['cash_flows_comp'].median()
  table1A[i]['earnings_mean'] = tempDF['earnings_comp'].mean()
  table1A[i]['earnings_median'] = tempDF['earnings_comp'].median()
print (table1A)

Table 1B

In [ ]:
### Replicate Table 1B ###
mainDF = tempMain

In [ ]:

#Initialize empty DataFrames
table1B = pd.DataFrame(columns = range (1, 11))
annualRetMinusRf = pd.DataFrame(index= range (2018, 2022), columns= range (1, 11))
for i in annualRetMinusRf.index:
  for j in annualRetMinusRf.columns:
    annualRetMinusRf[j][i]=[]


sizeofPortfolios = pd.DataFrame(index= ['size', 'size_mean', 'size_median'], columns= range(1,11))
for i in sizeofPortfolios.index:
  for i in sizeofPortfolios.columns:
    sizeofPortfolios[j][i]=[]

In [ ]:

#Calculate compound ret
for i in table1B.columns:
  tempDF = mainOnFYearEndDecOnly[mainOnFYearEndDecOnly['accruals_comp_decile'] == i]
  for i in tempDF. index:
    if((j - 11) in mainDF.index) and (mainDF['tic'][j - 11] == tempDF['tic'][j]): #check if index exists and is the same firm as current data
      tempDF2 = mainDF.loc[(j - 11) : j]
      tempDF2['ret _minus_rf_compounded'] = (1 + tempDF2['ret_minus_rf']).cumprod()-1
      yearofData = tempDF2['year'][j]
      annualRetMinusRf[1][yearofData].append(tempDF2['ret_minus_rf_compounded'][j]) #add last count of compounded data to annual ret minus Rf data
      sizeofPortfolios[i]['size'].append(np.log(tempDF2['mv'][j]))


In [ ]:
#Take average for each portfolio
for i in annualRetMinusRf.index:
  for i in annualRetMinusRf.columns:
    annualRetMinusRf[j][i] = np.average(annualRetMinusRf[j][i])

#new calculated comlum
# sizeofPortfolios['size_mean']= sizeofPortfolios['size'].lambda(x: np.average(x))


for j in sizeofPortfolios.columns:
  sizeofPortfolios[j]['size_mean'] = np.average(sizeofPortfolios[j]['size'])
  sizeofPortfolios[j]['size_median'] = np.median(sizeofPortfolios[j]['size'])

sizeofPortfolios.drop('size', inplace = True, axis=0)
#set index
annualRetMinusRf['year']= annualRetMinusRf.index
print(sizeofPortfolios)

In [ ]:
#Generate Dataframe for Table 18 Regresssions
dataForTable1BReg = annualRetMinusRf.merge(annualRmMinusRf,on='year', how = 'left')
dataForTable1BReg = dataforTable1BReg.astype(float)


#Run regressions to get alpha and beta
for i in range(1, 11):
  tempDF = dataforTable1BReg.filter([i, 'annual_rm_minus_rf'])
  tempDF.columns = ['current_decile_data','annual_rm_minus_rf']
  #making a line of best fit
  tempReg = sm.ols(formula = "current_decile_data ~ annual_rm_minus_rf", data = tempDF).fit()
  tempRegSummary = tempReg.summary()
  #printing components
  print("Accruals decile =" + str(i) + ":")
  print(tempRegSummary)

Table 1C

In [ ]:
mainDF= tempMain

In [ ]:
#Generate summary statistics (Table 1C)
#structure: rows with index names, 10 columns listed 1,2... 10 with stats
table1C= pd.Dataframe(index = ['current_asset_mean', 'current_asset_median', 'current_liability_mean', 'current_liability_median', 'depreciation_expense_mean', 'depreciation_expense_median'], columns = range(1, 11))
for i in table1C.columns:
  tempDF = mainOnFYearEnd[mainOnFYearEnd['accruals_comp_decile'] == i]
  table1C[i]['current_asset_mean mean'] = tempDF['accruals_subcomp_1'].mean()
  table1C[i]['current_asset_median'] = tempDF['accruals_subcomp_1'].median()
  table1C[i]['current_liability_mean']= tempDF['cash_flows_subcomp_2'].mean()
  table1C[i]['current_liability_median'] = tempDF['cash_flows_subcomp_2'].median()
  table1C[i]['depreciation_expense_mean'] = tempDF['earnings_subcomp_3'].mean()
  table1C[i]['depreciation_expense_median'] = tempDF['earnings_subcomp_2'].median()
print (table1C)

Table 2

In [ ]:
print(tempMain)

In [ ]:
#14

#Regenerate main on Year End (to allow us to rerun the same code block for debugging)
#generate quantile (precise number), but cut off to not be too long. multiply 0.1, 0.4, etc by 10 to get 1, 2, 3, etc.

mainDF= tempMain

mainOnFYearEnd = mainDF.dropna()

mainOnFYearEnd['accruals_comp_quantile'] = mainOnFYearEnd['accurals_comp'].rank(pct = True)

mainOnFYearEnd['accruals_comp_decile']= np.ceil(mainOnFYearEnd['accruals_comp_quantile'] * 10)

mainOnFYearEnd['earnings_comp_quantile'] = mainOnFYearEnd['earnings_comp'].rank(pct=True)

mainOnFYearEnd['earnings_comp_decile']= np.ceil(mainOnFYearEnd['earnings_comp_quantile'] * 10)

mainOnFYearEnd['cash_flows_comp_quantile']= mainOnFYearEnd['cash_flows_comp'].rank(pct = True)

mainOnFYearEnd['cash_flows_comp_decile']= np.ceil(mainOnFYearEnd['cash_flows_comp_quantile'] * 10)


print(mainOnFYearEnd)

In [ ]:
#Generate comp for next year's earnings component and accruals decile
#.shift function ask chat gpt
mainOnFYearEnd['accruals_comp_next_year']= mainOnFYearEnd.groupby(['tic'])['accruals_comp'].shift(-1)
mainOnFYearEnd['earnings_comp_next_year'] = mainOnFYearEnd.groupby(['tic'])['earnings_comp'].shift(-1)
mainOnFYearEnd['cash_flows_comp_next_year'] = mainOnFYearEnd.groupby(['tic'])['cash_flows_comp'].shift(-1)
mainOnFYearEnd['accruals_comp_decile_next_year']= mainOnFYearEnd.groupby(['tic'])['accruals_comp_decile'].shift(-1)
mainOnFYearEnd['earnings_comp_decile_next_year']= mainOnFYearEnd.groupby(['tic'])['earnings_comp_decile'].shift(-1)
mainOnFYearEnd['cash_flows_comp_decile_next_year']= mainOnFYearEnd.groupby(['tic'])['cash_flows_comp_decile'].shift(-1)

print(mainOnFYearEnd)

In [ ]:
#Generate DataFrame for table 2 regressions (drops most recent years to available text-year data)
dataForTable2Reg= mainOnFYearEnd.dropna()

print(dataForTable2Reg)

In [ ]:
#run regressions

table2AReg= sm.ols(formula= "earnings_comp_next_year ~ earnings_comp", data = dataForTable2Reg).fit()

table2ARegSummary= table2AReg.summary()

table2BReg = sm.ols(formula= "earnings_comp_decile_next_year~ earnings_comp_decile", data = dataForTable2Reg).fit()
table2BRegSummary= table2BReg.summary()


In [ ]:

#run regressions for firm grouped by industry codes
alpha0DistTable2A= []
alpha0DistTable2B= []

alpha1DistTable2A= []
alpha1DistTable2B = []

#i dont think can optimize- can create array and drop all industry codes that are 1


for i in dataForTable2Reg['siccd'].unique():
  tempDF= dataForTable2Reg[dataForTable2Reg['siccd'] == i]
  regA = sm.ols(formula= "earnings_comp_next_year~ earnings_comp", data = tempDF).fit()
  #what step is this? for each identifier, have one regression with factor and other without- is this the interacton term?
  alpha0DistTable2A.append(regA.params[0])
  #0 is intercept, 1 is x-axis/ independent variable
  alpha1DistTable2A.append(regA.params[1])
  regB = sm.ols(formula= "earnings_comp_decile_next_year~ earnings_comp_decile", data = tempDF).fit()
  alpha0DistTable2B.append(regB.params[0])
  alpha1DistTable2B.append(regB.params[1])


In [ ]:

#Print("Table 2A").results

#how to format the printing of graphs
print("Table 2A")
print("Pooled Regressions")
print(table2ARegSummary)
print("Industry Level Regressions")
#alpha of firms with indisutry code 0, industry code 1 seprately
print("Mean - alpha_0:" + str(np.average(alpha0DistTable2A)) + "alpha_1:" + str(np.average(alpha1DistTable2A)))

#what is comma seperating?
print("Q1 - alpha_0:" + str(np.quantile(alpha0DistTable2A, 0.25)) + "alpha_1:" + str(np.quantile(alpha1DistTable2A, 0.25)))

print("Median - alpha_0:" + str(np.median(alpha0DistTable2A)) + "alpha_1" (np.median(alpha1DistTable2A)))

print("Q3 - alpha_0:"+ str(np.quantile(alpha0DistTable2A, 0.75)) + "alpha_1": str(np.quantile(alpha1DistTable2A, 0.75)))



In [ ]:
#Print Table 28 results

print("Table 2B")
print("Pooled Regressions")
print(table2BRegSummary)
print("Industry Level Regressions")
print("Mean-alpha_0:" + str(np.average(alpha0DistTable2B))+ "alpha_1:" + str(np.average(alpha1DistTable2B)))
print("Q1 - alpha_0:" + str(np.quantile(alpha0DistTable2B, 0.25)) + "alpha_1:" + str(np.quantile(alpha1DistTable2B, 0.25)))
print("Median-alpha_0:"+ str(np.median(alpha0DistTable2B)) + "alpha_1:"+ str(np.median(alpha1DistTable2B)))
print("Q3 - alpha_0:"+ str(np.quantile(alpha0DistTable2B, 0.75))+ "alpha_1:" str(np.quantile(alpha1DistTable2B, 0.75)))


Table 3

In [ ]:
tempMainOnFYearEnd= mainOnFYearEnd

In [ ]:
dataforTable3Reg= dataforTable2Reg

In [ ]:
#Run regressions

table3AReg= sm.ols(formula= "earnings_comp_next_year~ accruals_comp + cash_flows_comp", data= dataforTable3Reg).fit()
table3ARegSummary= table3AReg.summary()
table3BReg= sm.ols(formula= "earnings_comp_decile_next_year~ accruals_comp_decile + cash_flows_comp_decile", data= dataforTable3Reg).fit()
table3BRegSummary= table3BReg.summary()

#gamma vs alpha
gamma0DistTable3A= []
gamma1DistTable3A= []
gamma2DistTable3A= []
gamma0DistTable3B= []
gamma1DistTable3B= []
gamma2DistTable3B= []


#run regressions for is grouped by industry codes
for i in dataforTable3Reg['siccd'].unique():
  #select specific column
  tempDF= dataforTable3Reg[dataforTable3Reg['siccd']== i]
  regA= sm.ols(formula= "earnings_comp_next_year, accruals_comp + cash_flows_comp", data = tempDF).fit()
  gamma0DistTable3A.append(regA.params[0])
  gamma1DistTable3A.append(regA.params[1])
  gamma2DistTable3A.append(regA.params[2])
  regB= sm.ols(formula= "earnings_comp_decile_next_year, accruals_comp_decile + cash_flows_comp_decile", data = tempDF).fit()
  gamma0DistTable3B.append(regB.params[0])
  gamma1DistTable3B.append(regB.params[1])
  gamma2DistTable3B.append(regB.params[2])

#Remove min and max values
gamma0DistTable3A.sort()
#what doing here
gamma0DistTable3A = gamma0DistTable3A[1: -1]

gamma1DistTable3A.sort()
gamma1DistTable3A = gamma1DistTable3A[1: -1]

gamma2DistTable3A.sort()
gamma2DistTable3A = gamma2DistTable3A[1: -1]

gamma0DistTable3B.sort()
gamma0DistTable3B = gamma0DistTable3B[1: -1]

gamma1DistTable3B.sort()
gamma1DistTable3B = gamma1DistTable3B[1: -1]

gamma2DistTable3B.sort()
gamma2DistTable3B = gamma2DistTable3B[1: -1]

table3ARegSummary= table3AReg.summary()

#Print Toole 34 results
print("Table 3A")
print("Pooled Regressions")
print(table3ARegSummary)
print("Industry Level Regressions")
print("Mean - gamma_0:" + str(np.average(gamma0DistTable3A))+ "gamma_1:" + str(np.average(gamma1DistTable3A))+ "gamma_2:" + str(np.average(gamma2DistTable3A)))
print("Q1 - gamma_0:" + str(np.quantile(gamma0DistTable3A, 0.25)) + "gamma_1:" + str(np.quantile(gamma1DistTable3A, 0.25))+ "gamma_2:" + str(np.quantile(gamma2DistTable3A, 0.25)))
print("Median - gamma_0:"+ str(np.median(gamma0DistTable3A)) + "gamma_1:"+ str(np.median(gamma1DistTable3A))+ "gamma_2:" + str(np.median(gamma2DistTable3A)))
print("Q3 - gamma_0:"+ str(np.quantile(gamma0DistTable3A, 0.75))+ "gamma_1:" str(np.quantile(gamma1DistTable3A, 0.75))+ "gamma_2:" + str(np.quantile(gamma2DistTable3A, 0.75)))


In [ ]:
#Print Toole 3B results
table3BRegSummary= table3BReg.summary()

print("Table 3B")
print("Pooled Regressions")
print(table3BRegSummary)
print("Industry Level Regressions")
print("Mean - gamma_0:" + str(np.average(gamma0DistTable3B))+ "gamma_1:" + str(np.average(gamma1DistTable3B))+ "gamma_2:" + str(np.average(gamma2DistTable3B)))
print("Q1 - gamma_0:" + str(np.quantile(gamma0DistTable3B, 0.25)) + "gamma_1:" + str(np.quantile(gamma1DistTable3B, 0.25))+ "gamma_2:" + str(np.quantile(gamma2DistTable3B, 0.25)))
print("Median - gamma_0:"+ str(np.median(gamma0DistTable3B)) + "gamma_1:"+ str(np.median(gamma1DistTable3B))+ "gamma_2:" + str(np.median(gamma2DistTable3B)))
print("Q3 - gamma_0:"+ str(np.quantile(gamma0DistTable3B, 0.75))+ "gamma_1:" str(np.quantile(gamma1DistTable3B, 0.75))+ "gamma_2:" + str(np.quantile(gamma2DistTable3B, 0.75)))




Prep for Table 4 Regression

In [ ]:
#18

### Replicate Table 4 ##

#Add compound abnormal return data for next year
#checkpoint
mainOnFYearEnd= tempMainOnFYearEnd
dataForTable4Reg= mainOnFYearEnd.dropna()
#initializing?
dataForTable4Reg['abn_ret_compounded_next_year'] = np.nan

for i in dataForTable4Reg.index:
  if((i + 4) in mainDF.index) and ((i + 15) in mainDF.index) and (mainDF['tic'][i + 4] == dataForTable4Reg['tic'][i]) and (mainDF['tic'][i + 15] == dataForTable4Reg['tic'][i]): #check if index for 4 months and 15 months after fiscal yer ends and exist and are the same firm as current data
    tempDF= mainDF.loc[(i+4)= (i + 15)]
    tempDF['abn_ret_compounded_next_year'] = (1+ tempDF['abn_ret']).cumprod() - 1
    dataForTable4Reg['abn_ret_compounded_next_year'][i]= tempDF['abn_ret_compounded_next_year'][i + 15] #add last count of compounded abnormal return data

dataForTable4Reg = dataForTable4Reg.dropna()

#Rank abnormal return data

dataForTable4Reg['abn_ret_compounded_quantile_next_year']= dataForTable4Reg['abn_net_compounded_next_year'].rank(pct = True)

dataForTable4Reg['abn_ret_compounded_decile_next_year'] = np.ceil(dataForTable4Reg['abn_ret_compounded_quantile_next_year'] * 10)

print(dataForTable4Reg)

In [ ]:
#19
#run regression
table4AReg1= sm.ols(formula= "earnings_comp_next_year~ earnings_comp", data= dataForTable4Reg).fit()
table4AReg1Summary= table4AReg1.summary()
print(table4AReg1Summary)

In [ ]:
#20
#Record data from previous regression
#parameter 0 is y-intercept, so y intercept is alpha
alpha_0_4A = table4AReg1.params[0]

dataForTable4Reg['earnings_comp_next_year_minus_alpha_0'] = dataForTable4Reg['earnings_comp_next_year']- alpha_0_4A

#Run second regression for Table 4A
table4AReg2= sm.ols(formula= "abn_ret_compounded_next_year ~ earnings_comp_next_year_minus_alpha_0 + earnings_comp -1", data= dataForTable4Reg).fit()
table4AReg2Summary= table4AReg2.summary()

print(table4AReg2Summary)

In [ ]:
#table 4b
table4BReg1= sm.ols(formula= "earnings_comp_decile_next_year ~ earnings_comp_decile", data= dataForTable4Reg).fit()
table4BReg1Summary= table4BReg1.summary()

print(table4BReg1Summary)

In [ ]:
alpha_0_4B = table4BReg1.params[0]

dataForTable4Reg['earnings_comp_decile_next_year_minus_alpha_0'] = dataForTable4Reg['earnings_comp_decile_next_year']- alpha_0_4B

#Run second regression for Table 4b
table4BReg2= sm.ols(formula= "abn_ret_compounded_decile_next_year ~ earnings_comp_decile_next_year_minus_alpha_0 + earnings_comp_decile -1", data= dataForTable4Reg).fit()
table4BReg2Summary= table4BReg2.summary()

print(table4BReg2Summary)

Table 5

In [ ]:
#23
#table 5
dataForTable5Reg= dataForTable4Reg

table5AReg1= sm.ols(formula= "earnings_comp_next_year ~ accruals_comp+ cash_flows_comp", data= dataForTable5Reg).fit()
table5AReg1Summary= table5AReg1.summary()

print(table5AReg1Summary)

alpha_0_5A = table5AReg1.params[0]

dataForTable5Reg['earnings_comp_next_year_minus_gamma_0'] = dataForTable5Reg['earnings_comp_next_year']- alpha_0_5A

#Run second regression for Table 4b
table5AReg2= sm.ols(formula= "abn_ret_compounded_next_year ~ earnings_comp_next_year_minus_gamma_0 + accruals_comp+ cash_flows_comp -1", data= dataForTable5Reg).fit()
table5AReg2Summary= table5AReg2.summary()

print(table5AReg2Summary)


In [ ]:
#table 5B
#25
table5BReg1= sm.ols(formula= "earnings_comp_decile_next_year ~ accruals_comp_decile+ cash_flows_comp_decile", data= dataForTable5Reg).fit()
table5BReg1Summary= table5BReg1.summary()

print(table5AReg1Summary)

alpha_0_5B = table5AReg1.params[0]

dataForTable5Reg['earnings_comp_decile_next_year_minus_gamma_0'] = dataForTable5Reg['earnings_comp_decile_next_year']- alpha_0_5B

#Run second regression for Table 4b
table5BReg2= sm.ols(formula= "abn_ret_compounded_decile_next_year ~ earnings_comp_decile_next_year_minus_gamma_0 + accruals_comp_decile+ cash_flows_comp_decile -1", data= dataForTable5Reg).fit()
table5BReg2Summary= table5BReg2.summary()

print(table5BReg2Summary)



Table 6

In [ ]:
#27]
### Replicate Table 6 ***

#Generate data (2-year abnormal return, ret-rf and rm-rf date)

dataForTable6= dataForTable5Reg

dataForTable6['abn_ret_compounded_next_2_years'] = np.nan
dataForTable6['ret_minus_rf_compounded_next_year'] = np.nan
dataForTable6['ret_minus_rf_compounded_next_2_years'] = np.nan
dataForTable6['rm_minus_rf_compounded_next_year'] = np.nan
dataForTable6['rm_minus_rf_compounded_next_2_years'] = np.nan

for i in dataForTable6.index
  if((i+4) in mainDF.index) and ((i+27) in mainDF.index) and (mainDF['tic'][i+4]==dataForTable6['tic'][i])and (mainDF['tic'][i+27] == dataForTable6['tic'][i]):
#check if index for 4 months and 27 months after fiscal yar ends and exists and is the same firm as current

#Add ret-rf, r-rfand abnormal return data
  tempDF = mainDF.loc[(i+4) :(i+27)]
#ret-rf
  tempDF['ret_minus_rf_compounded'] = (1 + tempDF['ret_minus_rf']).cumprod() - 1
  dataForTable6['ret_minus_rf_compounded_next_year'][i]= tempDF['ret_minus_rf_compounded'][i+15]
  dataForTable6['rm_minus_rf_compounded_next_2_years'][i]= tempDF['ret_minus_rf_compounded'][i+27]
#rm-rf
  tempDF['rm_minus_rf_compounded'] = (1 + tempDF['rm_minus_rf']).cumprod() - 1
  dataForTable6['rm_minus_rf_compounded_next_year'][i] = tempDF['rm_minus_rf_compounded'][i + 15]
  dataForTable6['rm_minus_rf_compounded_next_2_years'][i] = tempDF['rm_minus_rf_compounded'][i + 27]
#abnormal returns
  tempDF['abn_ret_compounded']= (1+ tempDF['abn_ret']).cumprod() -1
  dataForTable6['abn_ret_compounded_next_2_years'][i]= tempDF['abn_ret_compounded'][i + 27]
  dataForTable6 = dataForTable6.dropna()

table6 = pd.DataFrame(index = range(1,11), columns = ['sar_next_year_mean', 'sar_next_2 years_mean', 'sar_next_year_median', 'sar_next_2_years_median, 'alpha_next_year', 'alpha_next_2_years'])

for i in table6.index:
  tempDF = dataForTable6[dataForTables['accruals comp_decile'] =i]
  table6['sar_next_year_mean'][i]= np.average(tempDF['abn_ret_compounded_next_year'])
  table6['sar_next_2_years_mean'][i]= np.average(tempDF['abn_ret_compounded_next_2_years'])
  table6['sar_next_year_median'][1] = np.median(tempDF['abn_net_compounded_next_year'])
  table6['sar_next_2_years_median'][i]- np.median(tempDF['abn_ret_compounded_next_2_years'])

  tempReg= sm.ols(formula = "ret_minus_rf_compounded_next_year ~ rm_minus_rf_compounded_next_year", data= tempDF).fit()
  tempAlpha= tempReg.params[0]
  tempReg2= sm.ols(formula= "ret_minus_rm_compounded_next_2_years ~rm_minus_rf_compounded_next_2_years", data= tempDF).fit()
  tempAlpha2= tempReg2.params[0]
  table6['alpha_next_year'][i]= tempAlpha
  table6['alpha_next_2_years'][i]= tempAlpha2

print(table6)

Table 7

In [ ]:
#28
dataForTable7= dataForTable6
table7AReg1= sm.ols(formula= "abn_ret_compounded_next_year ~ accruals_comp", data= dataForTable7).fit()
table7AReg1Summary= table7AReg1.summary()
table7AReg2= sm.ols(formula= "abn_ret_compounded_next_2_years ~ accruals_comp", data= dataForTable7).fit()
table7AReg2Summary= table7AReg2.summary()

table7BReg1= sm.ols(formula= "abn_ret_compounded_next_year ~ accruals_subcomp_1+ accruals_subcomp_2+ accruals_subcomp_3", data= dataForTable7).fit()
table7BReg1Summary= table7BReg1.summary()
table7BReg2= sm.ols(formula= "abn_ret_compounded_next_2_years ~ accruals_subcomp_1+ accruals_subcomp_2+ accruals_subcomp_3", data= dataForTable7).fit()
table7BReg2Summary= table7BReg2.summary()

print(table7AReg1Summary)
print(table7AReg2Summary)
print(table7BReg1Summary)
print(table7BReg2Summary)